In [1]:
%matplotlib qt5
from process_files import *
from func_flats import *
from process_flat_fit import *

qt.qpa.xcb: X server does not support XInput 2


qt.qpa.xcb: QXcbConnection: XCB error: 1 (BadRequest), sequence: 169, resource id: 132, major code: 130 (Unknown), minor code: 47


In [2]:
line = 'Fe_I_6173'
fit_params1 = process_flat_fit(line, 'pol1')

Reading average flat files:  
 /arch/1/VTT/hellride/raw/20230502/L0/HELLRIDE_pol1_20230502_Flats_1309_Fe_I_6173_fa.FITS
Fit data written to:  
 /arch/1/VTT/hellride/raw/20230502/L0/HELLRIDE_pol1_20230502_Flats_1309_Fe_I_6173_ft.FITS


In [ ]:
# iline = 0
line = 'Fe_I_6173'
# calib data
config = configobj.ConfigObj('config.ini')
ffdir = config['flats']['directory']
settings = [f for f in os.listdir(ffdir) if 'settings' in f]
settings = ffdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)

In [ ]:
ff1name = config['flats'][line+'/pol1'].replace('_fm', '_fa')
ff2name = config['flats'][line+'/pol2'].replace('_fm', '_fa')
#
hdul = pf.open(ff1name)
ff1 = np.float64(hdul[0].data)
hdul = pf.open(ff2name)
ff2 = np.float64(hdul[0].data)
print('Reading average flat files: ', '\n', ff1name, '\n', ff2name)
#
fm1name = config['flats'][line+'/pol1'].replace('_fa', '_fm')
fm2name = config['flats'][line+'/pol2'].replace('_fa', '_fm')

In [ ]:
# Other numbers
linestr = 'Line_' + str(get_line_num(settings, line, iline))
nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
nwav = int(settings[linestr]['NWavePoints'])
filtstr = settings[linestr]['Filter']
modstr = settings[linestr]['Polarimeter\\Modulation']
nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
nfpc = nmod*nacc*nwav
nfpw = nmod*nacc
ncyc = len(os.listdir(ffdir+os.sep+line))//3
# Time stamps
tsfile = [f for f in os.listdir(ffdir) if 'timestamp' in f]
tsfile = ffdir + os.sep + tsfile[0]
ts = np.loadtxt(tsfile, delimiter=',')
im0ind, im1ind, im2ind = ts[:,3], ts[:,6], ts[:,9]

In [ ]:
Y, X = ff1.shape[0:2]
# ff1m = np.delete(ff1.reshape([Y,X,nacc*nmod,nwav], order='F'), 0, 2)
ff1m = ff1.reshape([Y,X,nacc*nmod,nwav], order='F')
ff1m = np.average(ff1m, axis=2)
# ff2m = np.delete(ff2.reshape([Y,X,nacc*nmod,nwav], order='F'), 0, 2)
ff2m = ff2.reshape([Y,X,nacc*nmod,nwav], order='F')
ff2m = np.average(ff2m, axis=2)

In [ ]:
fit_params1 = fit_lines_3d_parallel_poly2(ff1m, nparallel=16)
#
hdu1 = pf.PrimaryHDU(fit_params1)
hdul = pf.HDUList([hdu1])
hdul.writeto(ff1name, overwrite=True)
hdul.close()
#
config['flats'][line+'/pol1_fit'] = ff1name
config.write()

In [ ]:
fit_params2 = fit_lines_3d_parallel_poly2(ff2m, nparallel=16)
#
hdu1 = pf.PrimaryHDU(fit_params2)
hdul = pf.HDUList([hdu1])
hdul.writeto(ff2name, overwrite=True)
hdul.close()
#
config['flats'][line+'/pol2_fit'] = ff2name
config.write()

In [3]:
fig, axs = plt.subplots(2,3,figsize=(12,8))
for i, ax in enumerate(axs.flatten()):
    ax.imshow(fit_params1[:,:,i])
fig.tight_layout()

libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


In [ ]:
fit_params2.shape
fig, axs = plt.subplots(2,3,figsize=(12,8))
for i, ax in enumerate(axs.flatten()):
    ax.imshow(fit_params2[:,:,i])
fig.tight_layout()

In [ ]:
# Sample line fitting with predefined model
Y, X, Z = ff1m.shape
y, x  = 375, 30
line_data, wav_ind = ff1m[y,x,:], np.arange(Z)
# fit_res = fit_gauss_poly2_parallel(line_data, y, x)
fit_res = fit_gauss_poly2_parallel(line_data, y, x)
plt.plot(fit_res[2].best_fit, 'b')
# plt.plot(gauss_poly2(guess_params, wav_ind))
plt.plot(line_data, 'k--')
# plt.plot(line_data/gaussian_absroption_line(fit_params[3::], wav_ind))
# print(fit_res[2].best_values)
# res = fit_res[2]
plt.plot(fit_res[2].init_fit, 'r')
# plt.plot(line_data)
# plt.plot(res.init_fit)
# fit_res[2]

In [ ]:
def fit_gauss_poly2(spectral_line, line_width=5.0, gain_factor=1.0, plot=False):
    """
    Compute best fit for the line+continuum in the form of f(x)*[1-g(x)] where
    x is the pixel number starting at 0
    f(x) is quadratic (3 params)
    g(x) is Gaussian (3 params)
    Input:  1d-array spectral line
            (opt.) estimated width of the line
            (opt.) gain factor to estimate weights 
            (opt.) plot the results
    Output: list of inital guess parameters
            list of best fit parameters
    """
    xdata = np.arange(len(spectral_line))
    ydata = np.float64(spectral_line)
    ydata_error = np.sqrt(spectral_line.mean()/gain_factor)
    #
    deg0_guess = ydata[0]
    deg1_guess = (ydata[0]-ydata[-1])/(xdata[0]-xdata[-1])
    deg2_guess = 0
    amp_guess = 1-ydata.min()/deg0_guess
    cen_guess = xdata[np.argwhere(ydata==ydata.min())].flatten()[0]
    sig_guess = line_width/2.355
    #
    guess_params = Parameters()
    guess_params.add(name='amp', value=amp_guess, max=1)
    guess_params.add(name='cen', value=cen_guess, min=xdata.min(), max=xdata.max())
    guess_params.add(name='sig', value=sig_guess, min=0, max=xdata.max())
    guess_params.add(name='deg2', value=deg2_guess)
    guess_params.add(name='deg1', value=deg1_guess)
    guess_params.add(name='deg0', value=deg0_guess)
    #
    model = Model(gauss_poly2)
    result = model.fit(ydata, guess_params, xdata=xdata)
    return result
#
Y, X, Z = ff1m.shape
fit1_params = np.zeros([Y, X, 6])
for y in tqdm.tqdm(range(Y)):
    for x in range(X):
        res = fit_gauss_poly2(ff1m[y,x])
        fit1_params[y,x,:] = np.fromiter(res.best_values.values(), dtype=float)
hdu1 = pf.PrimaryHDU(fit1_params)
hdu2 = pf.ImageHDU(ff1m)
hdul = pf.HDUList([hdu1, hdu2])
hdul.writeto(ff1name, overwrite=True)
hdul.close()

In [ ]:
Y, X, Z = ff2m.shape
fit2_params = np.zeros([Y, X, 6])
for y in tqdm.tqdm(range(Y)) :
    for x in range(X):
        res = fit_gauss_poly2(ff2m[y,x])
        fit2_params[y,x,:] = np.fromiter(res.best_values.values(), dtype=float)
hdu1 = pf.PrimaryHDU(fit2_params)
hdu2 = pf.ImageHDU(ff2m)
hdul = pf.HDUList([hdu1, hdu2])
hdul.writeto(ff2name, overwrite=True)
hdul.close()

In [ ]:
# xdata = np.arange(ff1m.shape[2])
plt.imshow(fit1_params[:,:,0])
# plt.imshow(fit_line_shifts)
plt.colorbar()
# plt.clim([3,6])

In [ ]:
popt, fit_line_shifts = fit_2d_parabola(fit_params1[:,:,1])
plt.plot(fit_params1[600,:,1])
plt.plot(fit_line_shifts[600,:])

In [ ]:
# For photospheric lines
# Channel-1 fitting
guess_params1, fit_params1 = fit_lines_3d(ff1m)
continuum1, line_shifts1 = compute_line_params_3d(fit_params1, np.arange(Z))
#
hdu1 = pf.PrimaryHDU(continuum1)
hdu2 = pf.ImageHDU(line_shifts1)
hdu3 = pf.ImageHDU(ff1m)
hdul = pf.HDUList([hdu1, hdu2, hdu3])
hdul.writeto(ff1name, overwrite=True)
hdul.close()
# Channel-2 fitting
guess_params2, fit_params2 = fit_lines_3d(ff2m)
continuum2, line_shifts2 = compute_line_params_3d(fit_params2, np.arange(nwav))
#
hdu1 = pf.PrimaryHDU(continuum2)
hdu2 = pf.ImageHDU(line_shifts2)
hdu3 = pf.ImageHDU(ff2m)
hdul = pf.HDUList([hdu1, hdu2, hdu3])
hdul.writeto(ff2name, overwrite=True)
hdul.close()

In [ ]:
# For photospheric lines
# Channel-1 fitting
guess_params1, fit_params1 = fit_lines_3d(ff1m)
continuum1, line_shifts1 = compute_line_params_3d(fit_params1, np.arange(Z))
#
hdu1 = pf.PrimaryHDU(continuum1)
hdu2 = pf.ImageHDU(line_shifts1)
hdu3 = pf.ImageHDU(ff1m)
hdul = pf.HDUList([hdu1, hdu2, hdu3])
hdul.writeto(ff1name, overwrite=True)
hdul.close()
# Channel-2 fitting
guess_params2, fit_params2 = fit_lines_3d(ff2m)
continuum2, line_shifts2 = compute_line_params_3d(fit_params2, np.arange(nwav))
#
hdu1 = pf.PrimaryHDU(continuum2)
hdu2 = pf.ImageHDU(line_shifts2)
hdu3 = pf.ImageHDU(ff2m)
hdul = pf.HDUList([hdu1, hdu2, hdu3])
hdul.writeto(ff2name, overwrite=True)
hdul.close()

In [ ]:
config['flats'][line+'/bbi'] = ff0name
config['flats'][line+'/pol1'] = ff1name
config['flats'][line+'/pol2'] = ff2name
config.write()

In [ ]:
plt.imshow(line_shifts1)

In [ ]:
# To compute other artifacts, divide corrected flat array by template line profile 
ff1_corr = []
for i in tqdm.tqdm(range(ncyc)):
    dc1 = data_cube(ffdir, line, 1, i)
    temp1 = np.median(dc1.data.reshape([1280,1280,20,13], order='F'), axis=2) - dk1.data.reshape([Y,X,1])
    temp2 = shift_3darray(temp1/continuum, -line_shifts)
    ff1_corr.append(temp2)
ff1_cm = np.mean(np.array(ff1_corr)/ncyc, axis=0)
ff1_line = np.median(ff1_cm, axis=(0,1)).reshape([1,1,Z])
ff1_master = ff1_cm/ff1_line

In [ ]:
ff1_save = np.zeros([Y,X,2*Z+1])
ff1_save[:,:,0:Z] = continuum
ff1_save[:,:,Z] = line_shifts
ff1_save[:,:,Z+1::] = ff1_master

In [ ]:
# Compute line shifts using finding valleys

def compute_line_shifts(data, npix=1280):
    Y, X, Z = data.shape
    # crop data
    Ybeg, Yend, Xbeg, Xend = Y//2-npix//2, Y//2-npix//2+npix, X//2-npix//2, X//2-npix//2+npix
    data_ = data[Ybeg:Yend,Xbeg:Xend,:]
    line_cens = np.zeros([npix, npix])
    for y in tqdm.tqdm(range(npix)):
        for x in range(npix):
            lprof = data_[y,x,:]
            line_cens[y,x] = find_line_valley(lprof,npts=5)
    # Fit line shifts with a 2d quadratic (Poly2) equation
    YY, XX = np.meshgrid(np.arange(Ybeg,Yend), np.arange(Xbeg,Xend))
    x_ = np.vstack((YY.ravel(), XX.ravel()))
    p0 = (0,0,0,0,0)
    popt, pcov = curve_fit(poly2_2d, x_, line_cens.ravel(),p0=p0)
    # Extend the fit to the entire range
    YY, XX = np.meshgrid(np.arange(Y), np.arange(X))
    x_ = np.vstack((YY.ravel(), XX.ravel()))
    line_cens_fit = poly2_2d(x_, *popt).reshape([Y,X])
    return line_cens, line_cens_fit
# Plot the fits
line_cens, line_cens_fit = compute_line_shifts(ff1m)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
YY, XX = np.meshgrid(np.arange(Y), np.arange(X))
# ax.plot_surface(YY, XX, line_cens)
ax.plot_surface(YY, XX, line_cens_fit)
#

In [ ]:
plt.imshow(line_cens)

In [ ]:
from scipy.interpolate import interp1d
def find_valley_interp(line, res=0.01):
    """
    |   Compute line center by sub-pixel interpolation of Poly2 minima
    |   Input : 1d line profile, number of points around minima to consider for Poly2 interpolation 
    |   Output : valley position (double)
    """
    func = interp1d(np.arange(len(line)), line, kind='cubic')
    xnew = np.arange(0, len(line)-1, res)
    xmin = xnew[np.argmin(func(xnew))]
    return xmin

In [ ]:
# Compute line shifts using finding valleys
def compute_line_shifts(data):
    Y, X, Z = data.shape
    line_cens = np.zeros([Y, X])
    for y in tqdm.tqdm(range(Y)):
        for x in range(X):
            line_cens[y,x] = find_valley_interp(data[y,x,:])
    # Fit line shifts with a 2d quadratic (Poly2) equation
    YY, XX = np.meshgrid(np.arange(Y), np.arange(X))
    x_ = np.vstack((YY.ravel(), XX.ravel()))
    p0 = (0,0,0,0,0)
    popt, pcov = curve_fit(poly2_2d, x_, line_cens.ravel(),p0=p0)
    line_cens_fit = poly2_2d(x_, *popt).reshape([Y,X])
    return line_cens, line_cens_fit
# Plot the fits
line1_cens, line1_cens_fit = compute_line_shifts(ff1m)
line2_cens, line2_cens_fit = compute_line_shifts(ff2m)
#
fig = plt.figure()
ax1 = fig.add_subplot(211, projection='3d')
ax2 = fig.add_subplot(212, projection='3d')
YY, XX = np.meshgrid(np.arange(Y), np.arange(X))
ax1.plot_surface(YY, XX, line1_cens)
ax1.plot_surface(YY, XX, line1_cens_fit)
ax2.plot_surface(YY, XX, line2_cens)
ax2.plot_surface(YY, XX, line2_cens_fit)
#

In [ ]:
plt.plot(line1_cens[:,640])
plt.plot(line1_cens_fit[:,640])

In [ ]:
# Compute line shifts using finding valleys
Y, X, Z = ff2m.shape
line_cens = np.zeros([Y, X])
for y in tqdm.tqdm(range(Y)):
    for x in range(X):
        lprof = ff2m[y,x,:]
        line_cens[y,x] = find_line_valley(lprof)
# Fit line shifts with a 2d quadratic (Poly2) equation
x_ = np.tile(np.arange(Y).reshape([1,Y]),2).flatten()
YY, XX = np.meshgrid(np.arange(Y), np.arange(X))
x_ = np.vstack((YY.ravel(), XX.ravel()))
p0 = (0,0,0,0,0)
popt, pcov = curve_fit(poly2_2d, x_, line_cens.ravel(),p0=p0)
line_cens_fit = poly2_2d(x_, *popt).reshape([Y,X])
# Plot the fits
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
YY, XX = np.meshgrid(np.arange(Y), np.arange(X))
ax.plot_surface(YY, XX, line_cens)
ax.plot_surface(YY, XX, line_cens_fit)

In [ ]:
popt

In [ ]:
temp = line_cens[640]
temp[24:324] -= 0.46
plt.plot(line_cens[640])

In [ ]:
plt.plot(ff2m[640,322])
plt.plot(ff2m[640,336])

In [ ]:
plt.plot()

In [ ]:
# Shift all the lines to compute master
line_shifts = Z//2-line_cens_fit
ff1m_shifted = shift_3darray(ff1m, line_shifts)

In [ ]:
temp = ff1m_shifted/np.average(ff1m_shifted, axis=2).reshape([Y,X,1])
ltempl1 = np.median(temp, axis=(0,1)).reshape([1,1,Z])
# ltempl1m = np.mean(temp, axis=(0,1)).reshape([1,1,Z])
plt.figure()
plt.plot(ltempl1.flatten())
# plt.plot(ltempl1m.flatten())
ff1_gt = ff1m_shifted/ltempl1
continuum = ff1_gt/np.median(ff1_gt)